# Interaction with the system process

## Execute a command and capture the output


the following **only works with Python 3.7 and onwards:**

In [ ]:
import subprocess
result = subprocess.run(["ls", "-la"], capture_output=True)
print(result.stdout.decode("utf-8"))   # stdout is a bytestream, we need to decode it to utf-8

If you are (still) running **Python 3.6 or less** use this:

In [ ]:
import subprocess
result = subprocess.run(["ls", "-la"], stdout=subprocess.PIPE)
print(result.stdout.decode("utf-8"))   # stdout is a bytestream, we need to decode it to utf-8

**Note:** As of 2024, Python 3.6 is no longer maintained, so please make sure you upgrade to a newer version (currently: Python 3.12.x)


## Pipe the output of a command to another command 

In Unix, piping the results of one command into another is one of the biggest inventions since the invention of the surfboard. Unfortunately, in Python, piping is not very straight-forward to get it right:

1. start a subprocess p1 for the first command
2. retrieve the `stdout` from p1
3. start a subprocess p2, feed the `p1.stdout` into the `stdin` of p2
4. close the `stdout` of p1
5. invoke the `p2.communicate()` method
6. retrieve the output of p2

Here is a simple example which actually works, so you don't have to google it yourself.

In [ ]:
import subprocess

p1 = subprocess.Popen(["ls", "-la"], stdout=subprocess.PIPE)
p2 = subprocess.Popen(
    ["grep", "interaction"], 
    stdin=p1.stdout, 
    stdout=subprocess.PIPE
)
p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
output = p2.communicate()[0]
print(output.decode('utf-8'))


## Execute PowerShell scripts

This obviously only runs correctly if you're on a Windows machine or if you have PowerShell installed. First, we define a `run_powershell` function that runs a subprocess `pwsh`:

In [ ]:
import subprocess

def run_powershell(cmd):
    completed = subprocess.run(["pwsh", "-Command", cmd], capture_output=True)
    return completed

Then, we can use this function to run a hello-world. We need to call `.stdout.decode('utf-8')` to access the standard output of our subprocess and convert its bytestream into `utf-8`:

In [ ]:
hello_command = "Write-Host 'Hello World!'"
completed = run_powershell(hello_command)

print("Output was:", completed.stdout.decode('utf-8'))

If anything _can_ go wrong, it eventually _will_ go wrong. Make sure you always check the return code. If it is other than 0, something went wrong:

In [ ]:
bad_syntax_command = "Write-Hst 'Incorrect syntax command!'"
completed = run_powershell(bad_syntax_command)

if completed.returncode != 0:
    print("An error occured:", completed.stderr.decode('utf-8'))
else:
    print("Bad syntax command executed successfully!")

## Exercise 1

- [ ] play with the examples above, modify them to fit your ideas and needs
- [ ] remove the `.decode('utf-8')` method in the first subprocess example and observe the output. What does it mean?
- [ ] Replace `.decode('utf-8')` with `.decode('utf-16')` and run it again
- [ ] create a file containing an Umlaut (e.g. `äöü`). Replace `decode('utf-8')` with `decode('latin-1')` and observe the output 